In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [3]:
conf = SparkConf()
spark_context = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/25 18:22:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
spark = SparkSession.builder.master("spark://spark-master:7077").config("spark.jars.packages", 
                                                                        "org.apache.hadoop:hadoop-aws-2.7.3").appName("spark-example").getOrCreate()


26/01/25 19:38:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [10]:
print(spark.version)

3.5.5


In [7]:
import os
os.getcwd()
os.listdir()

['data', 'Untitled.ipynb', '.ipynb_checkpoints', 'notebooks']

In [9]:
csv_file = "/opt/workspace/data/oscars.csv"

df_csv = spark.read.csv(csv_file, header=True, inferSchema=True)
df_csv.show()


+--------+--------+------------+-------------------+--------------------+---+-------------+----------+--------+-------+-------+
|oscar_no|oscar_yr|       award|               name|               movie|age|     birth_pl|birth_date|birth_mo|birth_d|birth_y|
+--------+--------+------------+-------------------+--------------------+---+-------------+----------+--------+-------+-------+
|       1|    1929|Best actress|       Janet Gaynor|          7th Heaven| 22| Pennsylvania|1906-10-06|      10|      6|   1906|
|       2|    1930|Best actress|      Mary Pickford|            Coquette| 37|       Canada|1892-04-08|       4|      8|   1892|
|       3|    1931|Best actress|      Norma Shearer|        The Divorcee| 28|       Canada|1902-08-10|       8|     10|   1902|
|       4|    1932|Best actress|     Marie Dressler|        Min and Bill| 63|       Canada|1868-11-09|      11|      9|   1868|
|       5|    1933|Best actress|        Helen Hayes|The Sin of Madelo...| 32|Washington DC|1900-10-10|  

In [13]:
actor_df = spark.read.csv('/opt/workspace/data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('/opt/workspace/data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('/opt/workspace/data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('/opt/workspace/data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('/opt/workspace/data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('/opt/workspace/data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('/opt/workspace/data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('/opt/workspace/data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('/opt/workspace/data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('/opt/workspace/data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('/opt/workspace/data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('/opt/workspace/data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('/opt/workspace/data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('/opt/workspace/data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('/opt/workspace/data/store.csv', header=True, inferSchema=True)

In [23]:
#film_category_df.show()

In [ ]:
1. Вивести кількість фільмів в кожній категорії. Результат відсортувати за спаданням.

In [16]:
film_category_df.groupBy("category_id") \
      .agg(F.count("film_id").alias("num_films")) \
      .orderBy("category_id") \
      .show()

+-----------+---------+
|category_id|num_films|
+-----------+---------+
|          1|       64|
|          2|       66|
|          3|       60|
|          4|       57|
|          5|       58|
|          6|       68|
|          7|       62|
|          8|       69|
|          9|       73|
|         10|       61|
|         11|       56|
|         12|       51|
|         13|       63|
|         14|       61|
|         15|       74|
|         16|       57|
+-----------+---------+



In [ ]:
2.Вивести 10 акторів, чиї фільми брали на прокат найбільше. Результат відсортувати за спаданням.

In [26]:
# Для невеликих таблиць (actor, film_actor) можна зробити broadcast
from pyspark.sql.functions import broadcast

df = (rental_df
      .join(broadcast(inventory_df), "inventory_id")
      .join(broadcast(film_actor_df), "film_id")
      .join(broadcast(actor_df), "actor_id")
     )

result_df = (df.groupBy("actor_id", "first_name", "last_name")
               .agg(F.count("rental_id").alias("rental_count"))
               .orderBy(F.desc("rental_count"))
               .limit(10)
            )

result_df.show()

+--------+----------+-----------+------------+
|actor_id|first_name|  last_name|rental_count|
+--------+----------+-----------+------------+
|     107|      GINA|  DEGENERES|         753|
|     181|   MATTHEW|     CARREY|         678|
|     198|      MARY|     KEITEL|         674|
|     144|    ANGELA|WITHERSPOON|         654|
|     102|    WALTER|       TORN|         640|
|      60|     HENRY|      BERRY|         612|
|     150|     JAYNE|      NOLTE|         611|
|      37|       VAL|     BOLGER|         605|
|      23|    SANDRA|     KILMER|         604|
|      90|      SEAN|    GUINESS|         599|
+--------+----------+-----------+------------+



In [ ]:
3. Вивести категорія фільмів, на яку було витрачено найбільше грошей в прокаті

In [28]:
# Використовуємо broadcast для малих таблиць (category, film_category, inventory)
df = (payment_df
      .join(rental_df, "rental_id")                         # rental може бути велика → без broadcast
      .join(broadcast(inventory_df), "inventory_id")        # inventory зазвичай невелика
      .join(broadcast(film_category_df), "film_id")         # film_category невелика
      .join(broadcast(category_df), "category_id")          # category дуже маленька
     )

result_df = (df.groupBy("name")   # назва категорії
               .agg(F.round(F.sum("amount"),2).alias("total_amount"))
               .orderBy(F.desc("total_amount"))
               .limit(1)
            )

result_df.show()

+------+------------+
|  name|total_amount|
+------+------------+
|Sports|     5314.21|
+------+------------+



In [ ]:
4.Вивести назви фільмів, яких не має в inventory.

In [29]:
# left join film -> inventory
df = film_df.join(inventory_df, film_df.film_id == inventory_df.film_id, "left")

# залишаємо тільки ті фільми, яких немає в inventory
result_df = df.filter(inventory_df.film_id.isNull()).select(film_df.title)

result_df.show()


+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



In [ ]:
5.Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [30]:
# 1. Фільтруємо категорію "Children" одразу
children_category_df = category_df.filter(category_df.name == "Children")

# 2. Робимо joins
df = (rental_df
      .join(broadcast(inventory_df), "inventory_id")           # inventory зазвичай невелика → broadcast
      .join(broadcast(film_actor_df), "film_id")              # film_actor невелика → broadcast
      .join(broadcast(actor_df), "actor_id")                 # actor мала → broadcast
      .join(broadcast(film_category_df), "film_id")          # film_category невелика → broadcast
      .join(broadcast(children_category_df), "category_id")  # тільки Children
     )

# 3. Групуємо і рахуємо
result_df = (df.groupBy("first_name", "last_name")
               .agg(F.count("rental_id").alias("rental_count"))
               .orderBy(F.desc("rental_count"))
               .limit(3)
            )

result_df.show()

+----------+---------+------------+
|first_name|last_name|rental_count|
+----------+---------+------------+
|     HELEN|   VOIGHT|          97|
|    WHOOPI|     HURT|          86|
|      JADA|    RYDER|          84|
+----------+---------+------------+



In [31]:
spark.stop()